A Predictive Analysis of Heart Disease 

##Given a data set with various measurements of variables correlated to heart disease we are taking the given data to make a predictive data frame. This data frame will allow us to take a set of unknown values and predict whether or not it could indicate the development of heart disease in an individual. The question we are looking to answer is when given a set of unknown values we are able to predict the class of heart disease 

In [1]:
library(tidyverse)
library(repr)
library(readxl)
source("tests.R")
source("cleanup.R")
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [22]:
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleve.mod", destfile = "cleveland_data")
heart_disease_data <- read_delim("group_proposal/cleveland_data", col_names = FALSE, skip=20, delim= ",")
heart_disease_data

Rows: 283 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
64,1,1,110,211,0,2,144,1,1.8,2,0.0,3.0,0
58,0,1,150,283,1,2,162,0,1.0,1,0.0,3.0,0
58,1,2,120,284,0,2,160,0,1.8,2,0.0,3.0,1
58,1,3,132,224,0,2,173,0,3.2,1,2.0,7.0,3
60,1,4,130,206,0,2,132,1,2.4,2,2.0,7.0,4
50,0,3,120,219,0,0,158,0,1.6,2,0.0,3.0,0
58,0,3,120,340,0,0,172,0,0.0,1,0.0,3.0,0
66,0,1,150,226,0,0,114,0,2.6,3,0.0,3.0,0
43,1,4,150,247,0,0,171,0,1.5,1,0.0,3.0,0


In [51]:
names(heart_disease_data) <- c("age", "sex", "chest_pain", "resting_bp", "cholesterol", "fasting_blood_sugar", "resting_electro", "max_hr", "ex_angina", "oldpeak", "slope", "ca", "thal", "num")
heart_disease_data

age,sex,chest_pain,resting_bp,cholesterol,fasting_blood_sugar,resting_electro,max_hr,ex_angina,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
64,1,1,110,211,0,2,144,1,1.8,2,0.0,3.0,0
58,0,1,150,283,1,2,162,0,1.0,1,0.0,3.0,0
58,1,2,120,284,0,2,160,0,1.8,2,0.0,3.0,1
58,1,3,132,224,0,2,173,0,3.2,1,2.0,7.0,3
60,1,4,130,206,0,2,132,1,2.4,2,2.0,7.0,4
50,0,3,120,219,0,0,158,0,1.6,2,0.0,3.0,0
58,0,3,120,340,0,0,172,0,0.0,1,0.0,3.0,0
66,0,1,150,226,0,0,114,0,2.6,3,0.0,3.0,0
43,1,4,150,247,0,0,171,0,1.5,1,0.0,3.0,0
